In [29]:
import cv2
import numpy as np
import open3d as o3d
import json
import glob
import matplotlib.pyplot as plt
import copy
import teaserpp_python
from numpy.linalg import inv
from scipy.spatial import cKDTree
import time
from PIL import Image
import torchvision
import torch
from torchvision import transforms as T 
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from numba import jit, prange
import os
import csv 
# Load MaskRCNN 

def register_two_views_teaser(A_pcd_raw,B_pcd_raw,VOXEL_SIZE):
    
    VISUALIZE = True
    A_pcd = A_pcd_raw.voxel_down_sample(voxel_size=VOXEL_SIZE)
    B_pcd = B_pcd_raw.voxel_down_sample(voxel_size=VOXEL_SIZE)
    #if VISUALIZE:
     #   o3d.visualization.draw_geometries([A_pcd,B_pcd]) # plot downsampled A and B 

    A_xyz = pcd2xyz(A_pcd) # np array of size 3 by N
    B_xyz = pcd2xyz(B_pcd) # np array of size 3 by M

    print("Extracting FPFH features")
    # extract FPFH features
    A_feats = extract_fpfh(A_pcd,VOXEL_SIZE)
    B_feats = extract_fpfh(B_pcd,VOXEL_SIZE)
    print(A_feats.shape)
    print("Computing FPFH correspondences")
    # establish correspondences by nearest neighbour search in feature space
    corrs_A, corrs_B = find_correspondences(
        A_feats, B_feats, mutual_filter=True)
    A_corr = A_xyz[:,corrs_A] # np array of size 3 by num_corrs
    B_corr = B_xyz[:,corrs_B] # np array of size 3 by num_corrs

    num_corrs = A_corr.shape[1]
    print(f'FPFH generates {num_corrs} putative correspondences.')

    # visualize the point clouds together with feature correspondenc
    # robust global registration using TEASER++
    NOISE_BOUND = VOXEL_SIZE
    teaser_solver = get_teaser_solver(NOISE_BOUND)
    teaser_solver.solve(A_corr,B_corr)
    solution = teaser_solver.getSolution()
    R_teaser = solution.rotation
    t_teaser = solution.translation
    T_teaser = Rt2T(R_teaser,t_teaser)

    # Visualize the registration results
    A_pcd_T_teaser = copy.deepcopy(A_pcd).transform(T_teaser)
    #o3d.visualization.draw_geometries([A_pcd_T_teaser,B_pcd])

    # local refinement using ICP
    icp_sol = o3d.pipelines.registration.registration_icp(
          A_pcd, B_pcd, NOISE_BOUND, T_teaser,
          o3d.pipelines.registration.TransformationEstimationPointToPoint(),
          o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=100))
    T_icp = icp_sol.transformation

    # visualize the registration after ICP refinement
    A_pcd_T_icp = copy.deepcopy(A_pcd).transform(T_icp)
    if VISUALIZE:
        Acopy = copy.deepcopy(A_pcd_T_icp).paint_uniform_color([0.0,0.0,1])
        Bcopy = copy.deepcopy(B_pcd).paint_uniform_color([1.0,0.0,0.0])
        o3d.visualization.draw_geometries([Acopy,Bcopy])
    tformed_A = copy.deepcopy(A_pcd_raw).transform(T_icp)
    res = o3d.geometry.PointCloud()
    res = tformed_A + B_pcd_raw
    
    return res,T_icp

def pcd2xyz(pcd):
    return np.asarray(pcd.points).T

def extract_fpfh(pcd, voxel_size):
    radius_normal = voxel_size * 2
    pcd.estimate_normals(
      o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    fpfh = o3d.pipelines.registration.compute_fpfh_feature(
      pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return np.array(fpfh.data).T

def find_knn_cpu(feat0, feat1, knn=1, return_distance=False):
    feat1tree = cKDTree(feat1)
    dists, nn_inds = feat1tree.query(feat0, k=knn, workers=10)
    if return_distance:
        return nn_inds, dists
    else:
        return nn_inds

def find_correspondences(feats0, feats1, mutual_filter=True):
    nns01 = find_knn_cpu(feats0, feats1, knn=1, return_distance=False)
    corres01_idx0 = np.arange(len(nns01))
    corres01_idx1 = nns01

    if not mutual_filter:
        return corres01_idx0, corres01_idx1

    nns10 = find_knn_cpu(feats1, feats0, knn=1, return_distance=False)
    corres10_idx1 = np.arange(len(nns10))
    corres10_idx0 = nns10

    mutual_filter = (corres10_idx0[corres01_idx1] == corres01_idx0)
    corres_idx0 = corres01_idx0[mutual_filter]
    corres_idx1 = corres01_idx1[mutual_filter]

    return corres_idx0, corres_idx1

def get_teaser_solver(noise_bound):
    solver_params = teaserpp_python.RobustRegistrationSolver.Params()
    solver_params.cbar2 = 1.0
    solver_params.noise_bound = noise_bound
    solver_params.estimate_scaling = False
    solver_params.inlier_selection_mode = \
        teaserpp_python.RobustRegistrationSolver.INLIER_SELECTION_MODE.PMC_EXACT
    solver_params.rotation_tim_graph = \
        teaserpp_python.RobustRegistrationSolver.INLIER_GRAPH_FORMULATION.CHAIN
    solver_params.rotation_estimation_algorithm = \
        teaserpp_python.RobustRegistrationSolver.ROTATION_ESTIMATION_ALGORITHM.GNC_TLS
    solver_params.rotation_gnc_factor = 1.4
    solver_params.rotation_max_iterations = 10000
    solver_params.rotation_cost_threshold = 1e-16
    solver = teaserpp_python.RobustRegistrationSolver(solver_params)
    return solver

def Rt2T(R,t):
    T = np.identity(4)
    T[:3,:3] = R
    T[:3,3] = t
    return T 

def save_results(animal_id, data_path, mesh, pcd_downsampled, sa, v):
    # Save the mesh and PCD
    mesh_filename = os.path.join(data_path, f"{animal_id}_mesh.ply")
    pcd_filename = os.path.join(data_path, f"{animal_id}_pcd_downsampled.ply")

    o3d.io.write_triangle_mesh(mesh_filename, mesh)
    
    o3d.io.write_point_cloud(pcd_filename, pcd_downsampled)

    CSV_PATH = '/home/vigir3d/Datasets/cattle_scans/cow_measurements.csv'
    # Check if CSV file exists to decide whether to write headers
    write_header = not os.path.exists(CSV_PATH)
  # Save SA & V to the CSV file in append mode
    with open(CSV_PATH, 'a', newline='') as csvfile:
        fieldnames = ['Animal ID', 'SA', 'V']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        if write_header:
            writer.writeheader()

        writer.writerow({'Animal ID': animal_id, 'SA': sa, 'V': v})

        
@jit(nopython=True, parallel=True)
def numba_eliminate_flying_pixels(depth_image, ws, threshold):
    height, width = depth_image.shape
    result = np.zeros_like(depth_image, dtype=np.float64)
    
    for cy in prange(height):
        for cx in prange(width):
            x_start, x_end = max(0, cx - ws), min(width, cx + ws + 1)
            y_start, y_end = max(0, cy - ws), min(height, cy + ws + 1)
            window = depth_image[y_start:y_end, x_start:x_end]
            result[cy, cx] = np.sum(np.abs(window - depth_image[cy, cx]))
    
    return result


def load_maskrcnn_model(model_path):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, 2)
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, hidden_layer, 2)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    return model

def get_model_mask(model_generic, image):
    proba_threshold = 0.5
    ig = transform(image)
    with torch.no_grad():
        prediction = model_generic([ig.to(device)])
        
    if(prediction[0]['masks'].nelement() == 0):
        XX = torch.empty((0,0), dtype=torch.int64)
        return XX
    predicted_mask = prediction[0]
    predicted_mask = predicted_mask['masks'][0] > proba_threshold
    
    predicted_mask = predicted_mask.squeeze(1)
    mask = predicted_mask.cpu().detach()
    return mask

def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])
    
def segment_images_modified(last_frame):
    depth_image_array = np.asarray(last_frame.depth)
    print("Pre:",np.max(depth_image_array), " -- ", np.min(depth_image_array))
    depth_PIL = Image.fromarray(np.asarray(last_frame.depth)).convert("RGB")
    rgb_image_array = np.asarray(last_frame.color)
    rgb_PIL = Image.fromarray(rgb_image_array)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    rgb_mask = get_model_mask(model_rgb, rgb_PIL)
    depth_mask = get_model_mask(model_depth, depth_PIL)

    if depth_mask.nelement() == 0:
        mask_combined = rgb_mask
    else:
        mask_combined = depth_mask | rgb_mask  # 1 vote arbitration (OR the masks)

    # Convert tensor to numpy array and ensure the right datatype
    mask_combined = mask_combined.numpy().astype(rgb_image_array.dtype)
    mask_image = mask_combined.swapaxes(0, 2).swapaxes(0, 1)
    mask_image = (mask_image > 0).astype(rgb_image_array.dtype)   

    fg_image_rgb = rgb_image_array * mask_image

    # For the depth image:
    squeezed_mask = np.squeeze(mask_image)
    fg_image_depth = (depth_image_array * squeezed_mask)*1000 # upscale because re-inserting in "last_frame" rescales
    
    print("Post:" ,np.max(fg_image_depth), " -- ", np.min(fg_image_depth))
    last_frame.color = o3d.geometry.Image(fg_image_rgb)
    last_frame.depth = o3d.geometry.Image(fg_image_depth)

    return last_frame


def cluster_point_cloud_new(outlier_cloud):
    cloud_colors = copy.deepcopy(np.asarray(outlier_cloud.colors).T)
    
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        labels = np.array(outlier_cloud.cluster_dbscan(eps=0.04, min_points=10, print_progress=False))

    # Identify the largest cluster
    values, counts = np.unique(labels, return_counts=True)
    ind = np.argmax(counts)
    largest_cluster_label = values[ind]
    #print(f"Largest cluster label: {largest_cluster_label}")

    # Filter points, normals, and colors for the largest cluster
    cloud_xyz = pcd2xyz(outlier_cloud)
    cloud_normals = pcd2normals(outlier_cloud)

    cloud_filtered = cloud_xyz[:, labels == largest_cluster_label]
    normals_filtered = cloud_normals[:, labels == largest_cluster_label]
    colors_filtered = cloud_colors[:, labels == largest_cluster_label]

    # Create a point cloud for the largest cluster
    pcd_filtered_largest_cluster = o3d.geometry.PointCloud()
    pcd_filtered_largest_cluster.points = o3d.utility.Vector3dVector(cloud_filtered.T)
    pcd_filtered_largest_cluster.normals = o3d.utility.Vector3dVector(normals_filtered.T)
    pcd_filtered_largest_cluster.colors = o3d.utility.Vector3dVector(colors_filtered.T)

    #o3d.visualization.draw_geometries([pcd_filtered_largest_cluster])
    return pcd_filtered_largest_cluster

def load_calibration(tform_path, num_transforms=5):
    transforms = [np.eye(4)]  # Initialize with identity matrix
    
    # Load transformations from file
    for i in range(1, num_transforms+1):
        filename = tform_path + f"H_0_{i}.txt"
        
        if not os.path.exists(filename):
            raise FileNotFoundError(f"The file {filename} does not exist!")
        
        transforms.append(np.loadtxt(filename))
    
    return transforms

def upsample_using_reference_normals_new(sparse_pcd, dense_pcd, search_radius=0.02, angle_threshold=30):
    # Ensure the point clouds have normals
    if not sparse_pcd.has_normals():
        sparse_pcd.estimate_normals()
    if not dense_pcd.has_normals():
        dense_pcd.estimate_normals()
    
    # Ensure the point clouds have colors
    if not sparse_pcd.has_colors():
        sparse_pcd.paint_uniform_color([0.5, 0.5, 0.5])  # default gray color
    if not dense_pcd.has_colors():
        dense_pcd.paint_uniform_color([0.5, 0.5, 0.5])  # default gray color
    
    dense_points = np.asarray(dense_pcd.points)
    dense_tree = cKDTree(dense_points)
    
    sparse_points = np.asarray(sparse_pcd.points)
    sparse_normals = np.asarray(sparse_pcd.normals)
    sparse_colors = np.asarray(sparse_pcd.colors)
    
    # This retrieves the indices of all neighbors within the search_radius
    neighbor_indices = dense_tree.query_ball_point(sparse_points, search_radius, workers=-1)
    
    valid_indices = []
    for i, idx_row in enumerate(neighbor_indices):
        neighbors = dense_points[idx_row]
        neighbor_normals = np.asarray(dense_pcd.normals)[idx_row]
        neighbor_colors = np.asarray(dense_pcd.colors)[idx_row]
        
        # Calculate angles between sparse point's normal and all its neighbors' normals
        angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normals[i], neighbor_normals.T), -1.0, 1.0)))
        
        # Filtering neighbors based on angle threshold
        valid_neighbor_indices = np.array(idx_row)[angles < angle_threshold]
        valid_indices.extend(valid_neighbor_indices)
    
    unique_valid_indices = np.unique(valid_indices)
    final_valid_neighbors = dense_points[unique_valid_indices]
    final_valid_normals = np.asarray(dense_pcd.normals)[unique_valid_indices]
    final_valid_colors = np.asarray(dense_pcd.colors)[unique_valid_indices]
    
    upsampled_points = np.vstack([sparse_points, final_valid_neighbors])
    upsampled_normals = np.vstack([sparse_normals, final_valid_normals])
    upsampled_colors = np.vstack([sparse_colors, final_valid_colors])
    
    upsampled_pcd = o3d.geometry.PointCloud()
    upsampled_pcd.points = o3d.utility.Vector3dVector(upsampled_points)
    upsampled_pcd.normals = o3d.utility.Vector3dVector(upsampled_normals)
    upsampled_pcd.colors = o3d.utility.Vector3dVector(upsampled_colors)
    
    return upsampled_pcd


def process_file_list_comp(file_id, dpath):
    ws = 2
    # Format filenames
    dpath = dpath.rstrip('/')

    # Split by the directory separator and get the last part
    suffix = os.path.basename(dpath)
    
    file_suffix = suffix + "_nano_" + str(file_id)
    # Construct full file paths using os.path.join for better cross-platform compatibility
    depth_file = os.path.join(dpath, file_suffix + "_depth.png")
    color_file = os.path.join(dpath, file_suffix + ".jpg")
    intrinsics_file = os.path.join(dpath, file_suffix + "_intrinsics.txt")
    # Read the files
    depth = o3d.io.read_image(depth_file)
    color = o3d.io.read_image(color_file)
    #depth = o3d.io.read_image(dpath + 'Animal_box3_u_nano_11_depth.png')
    #color = o3d.io.read_image(dpath + 'Animal_box3_u_nano_11.jpg')
    K = np.loadtxt(intrinsics_file)
      

    camera_intrinsics = o3d.camera.PinholeCameraIntrinsic()
    camera_intrinsics.set_intrinsics(int(K[0]), int(K[1]), K[2], K[3], K[4], K[5])

    rgbdc = o3d.geometry.RGBDImage.create_from_color_and_depth(
        color, depth, depth_trunc=4.0, convert_rgb_to_intensity=False
    )

    volume = o3d.pipelines.integration.ScalableTSDFVolume(
        voxel_length=4.0 / 512.0, sdf_trunc=0.04, color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8
    )

    volume.integrate(rgbdc, camera_intrinsics, np.eye(4))
    pcd_tsdf = volume.extract_point_cloud()
    
    depth_np = np.asarray(rgbdc.depth)
    
    if ( depth_np.max() > 20 ) : # Depth is in millimeters
        flying_pixel_filter_threshold = 100 # 100
    else : # meters
        flying_pixel_filter_threshold = 0.1 # 0.1
        #print(flying_pixel_filter_threshold, " -- ", depth_np.max())

    result_mask = numba_eliminate_flying_pixels(depth_np.copy(), ws, flying_pixel_filter_threshold)
    depth_np[result_mask > flying_pixel_filter_threshold] = 0
    # Re-insert filtered depth into the rgbd image
    rgbdc.depth = o3d.geometry.Image(depth_np)
    # Apply maskrcnn to filtered depth image
    masked_rgbd = segment_images_modified(rgbdc)
    # necessary lol
    rgbdc_new = o3d.geometry.RGBDImage.create_from_color_and_depth(
                    masked_rgbd.color, masked_rgbd.depth, depth_trunc=4.0, convert_rgb_to_intensity=False)
    d = np.asarray(rgbdc_new.depth)
    print("Segmented depth | max : ", np.max(d), " min : ", np.min(d))

    v = o3d.pipelines.integration.ScalableTSDFVolume(
        voxel_length=4.0 / 512.0, sdf_trunc=0.04, color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8
    )

    v.integrate(rgbdc_new, camera_intrinsics, np.eye(4))
    pcd_tmp = v.extract_point_cloud()
    #pcd_tmp = o3d.geometry.PointCloud.create_from_rgbd_image(rgbdc_new,
     #                                                   camera_intrinsics)

    return pcd_tmp, pcd_tsdf

    
def perform_pairwise_alignment(pcds_tsdf,pcds_cropped):
    """Compute and apply transformations."""
    t01 = colored_ICP(pcds_tsdf[0], pcds_tsdf[1])
    t52 = colored_ICP(pcds_tsdf[5], pcds_tsdf[2])
    t43 = colored_ICP(pcds_tsdf[4], pcds_tsdf[3])
    t12 = colored_ICP(pcds_tsdf[1], pcds_tsdf[2])
    t32 = colored_ICP(pcds_tsdf[3], pcds_tsdf[2])

    H0 = t12 @ t01
    H1 = t12
    H3 = t32
    H4 = t32 @ t43
    H5 = t52

    # Transform the point clouds
    p0 = copy.deepcopy(pcds_cropped[0]).transform(H0)
    p1 = copy.deepcopy(pcds_cropped[1]).transform(H1)
    p3 = copy.deepcopy(pcds_cropped[3]).transform(H3)
    p4 = copy.deepcopy(pcds_cropped[4]).transform(H4)
    p5 = copy.deepcopy(pcds_cropped[5]).transform(H5)
    
    d0 = copy.deepcopy(pcds_tsdf[0]).transform(H0)
    d1 = copy.deepcopy(pcds_tsdf[1]).transform(H1)
    d3 = copy.deepcopy(pcds_tsdf[3]).transform(H3)
    d4 = copy.deepcopy(pcds_tsdf[4]).transform(H4)
    d5 = copy.deepcopy(pcds_tsdf[5]).transform(H5)
    
    pcd_combined = o3d.geometry.PointCloud()
    pcd_combined = p0+p1+pcds_cropped[2]+p3+p4+p5
    
    ptsdf_combined = o3d.geometry.PointCloud()
    ptsdf_combined = d0+d1+pcds_tsdf[2]+d3+d4+d5

    return pcd_combined, ptsdf_combined

    return  ptsdf_combined
def pcd2normals(pcd):
    return np.asarray(pcd.normals).T


In [ ]:
0.12*1000

In [28]:
path = '/home/vigir3d/Datasets/cattle_scans/Cattle_11_17_22/Animal_14_1/'
dpath = '/home/vigir3d/Datasets/cattle_scans/Cattle_11_17_22/Animal_1_1/'
file_ids = [11,12,14,16,17,18]  # exclude head cameras

# Generate a list of volumes using list comprehension
start = time.time()

results = [process_file_list_comp(file_id, path) for file_id in file_ids]
pcds,ptsdf = zip(*results)
end = time.time()
print("Load generate: " , end-start, "s")


#r01 = np.loadtxt(dpath+"r01.txt" )
#r12 = np.loadtxt(dpath+"r12_0031.txt")
#r32 = np.loadtxt(dpath+"r32_0026.txt")
#r52 = np.loadtxt(dpath+"r52_005.txt")
#r43 = np.loadtxt(dpath+"r43_0031.txt")

h0 = r12@r01
h1 = r12
h3 = r32
h4 = r32@r43
h5 = r52


a0 = copy.deepcopy(pcds[0]).transform(h0)
a1 = copy.deepcopy(pcds[1]).transform(h1)
a3 = copy.deepcopy(pcds[3]).transform(h3)
a4 = copy.deepcopy(pcds[4]).transform(h4)
a5 = copy.deepcopy(pcds[5]).transform(h5)

o3d.visualization.draw_geometries([a0,a1,a3,a4,a5,pcds[2]])

FileNotFoundError: /home/vigir3d/Datasets/cattle_scans/Cattle_11_17_22/Animal_14_1/Animal_14_1_nano_11_intrinsics.txt not found.

In [26]:

a0 = copy.deepcopy(pcds[0]).transform(h0)
a1 = copy.deepcopy(pcds[1]).transform(h1)
a3 = copy.deepcopy(pcds[3]).transform(h3)
a4 = copy.deepcopy(pcds[4]).transform(h4)
a5 = copy.deepcopy(pcds[5]).transform(h5)

o3d.visualization.draw_geometries([a0,a1,a3,a4,a5,pcds[2]])

In [10]:
pcd_f = o3d.geometry.PointCloud()
pcd_f = a0+a1+a3+a4+a5+pcds[2]
#o3d.io.write_point_cloud(dpath+"animal_1_1_ds.ply", pcd_f)

In [33]:
dist = 0.05
a = cluster_point_cloud_new(pcd_f,dist)
draw_registration_result(pcd_f,a,np.eye(4))

In [12]:
o3d.visualization.draw_geometries([a])

In [27]:
#p = upsample_using_reference_normals_new(a,pcd_a,0.02)
c = upsample_using_reference_normals_adaptable(a,pcd_a,0.02,0.1,30,100)
#draw_registration_result(p,a,np.eye(4))
draw_registration_result(c,a,np.eye(4))

In [20]:


def upsample_using_reference_normals_adaptable(sparse_pcd, dense_pcd, initial_search_radius=0.02, max_search_radius=0.1, angle_threshold=30, density_threshold=10):
    if not sparse_pcd.has_normals():
        sparse_pcd.estimate_normals()
    if not dense_pcd.has_normals():
        dense_pcd.estimate_normals()

    if not sparse_pcd.has_colors():
        sparse_pcd.paint_uniform_color([0.5, 0.5, 0.5])
    if not dense_pcd.has_colors():
        dense_pcd.paint_uniform_color([0.5, 0.5, 0.5])
    
    dense_points = np.asarray(dense_pcd.points)
    dense_tree = cKDTree(dense_points)
    
    sparse_points = np.asarray(sparse_pcd.points)
    sparse_normals = np.asarray(sparse_pcd.normals)
    sparse_colors = np.asarray(sparse_pcd.colors)

    valid_indices = []
    for i, sparse_point in enumerate(sparse_points):
        search_radius = initial_search_radius
        while True:
            neighbor_indices = dense_tree.query_ball_point(sparse_point, search_radius)
            if len(neighbor_indices) >= density_threshold or search_radius >= max_search_radius:
                break  # Sufficient density or max radius reached
            search_radius += 0.01  # Increment search radius

        neighbors = dense_points[neighbor_indices]
        neighbor_normals = np.asarray(dense_pcd.normals)[neighbor_indices]
        neighbor_colors = np.asarray(dense_pcd.colors)[neighbor_indices]

        angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normals[i], neighbor_normals.T), -1.0, 1.0)))

        valid_neighbor_indices = np.array(neighbor_indices)[angles < angle_threshold]
        valid_indices.extend(valid_neighbor_indices)
    
    unique_valid_indices = np.unique(valid_indices)
    final_valid_neighbors = dense_points[unique_valid_indices]
    final_valid_normals = np.asarray(dense_pcd.normals)[unique_valid_indices]
    final_valid_colors = np.asarray(dense_pcd.colors)[unique_valid_indices]

    upsampled_points = np.vstack([sparse_points, final_valid_neighbors])
    upsampled_normals = np.vstack([sparse_normals, final_valid_normals])
    upsampled_colors = np.vstack([sparse_colors, final_valid_colors])

    upsampled_pcd = o3d.geometry.PointCloud()
    upsampled_pcd.points = o3d.utility.Vector3dVector(upsampled_points)
    upsampled_pcd.normals = o3d.utility.Vector3dVector(upsampled_normals)
    upsampled_pcd.colors = o3d.utility.Vector3dVector(upsampled_colors)

    return upsampled_pcd



In [2]:
def upsample_using_reference_normals_new(sparse_pcd, dense_pcd, search_radius=0.02, angle_threshold=30):
    # Ensure the point clouds have normals
    if not sparse_pcd.has_normals():
        sparse_pcd.estimate_normals()
    if not dense_pcd.has_normals():
        dense_pcd.estimate_normals()
    
    # Ensure the point clouds have colors
    if not sparse_pcd.has_colors():
        sparse_pcd.paint_uniform_color([0.5, 0.5, 0.5])  # default gray color
    if not dense_pcd.has_colors():
        dense_pcd.paint_uniform_color([0.5, 0.5, 0.5])  # default gray color
    
    dense_points = np.asarray(dense_pcd.points)
    dense_tree = cKDTree(dense_points)
    
    sparse_points = np.asarray(sparse_pcd.points)
    sparse_normals = np.asarray(sparse_pcd.normals)
    sparse_colors = np.asarray(sparse_pcd.colors)
    
    # This retrieves the indices of all neighbors within the search_radius
    neighbor_indices = dense_tree.query_ball_point(sparse_points, search_radius, workers=-1)
    
    valid_indices = []
    for i, idx_row in enumerate(neighbor_indices):
        neighbors = dense_points[idx_row]
        neighbor_normals = np.asarray(dense_pcd.normals)[idx_row]
        neighbor_colors = np.asarray(dense_pcd.colors)[idx_row]
        
        # Calculate angles between sparse point's normal and all its neighbors' normals
        angles = np.degrees(np.arccos(np.clip(np.dot(sparse_normals[i], neighbor_normals.T), -1.0, 1.0)))
        
        # Filtering neighbors based on angle threshold
        valid_neighbor_indices = np.array(idx_row)[angles < angle_threshold]
        valid_indices.extend(valid_neighbor_indices)
    
    unique_valid_indices = np.unique(valid_indices)
    final_valid_neighbors = dense_points[unique_valid_indices]
    final_valid_normals = np.asarray(dense_pcd.normals)[unique_valid_indices]
    final_valid_colors = np.asarray(dense_pcd.colors)[unique_valid_indices]
    
    upsampled_points = np.vstack([sparse_points, final_valid_neighbors])
    upsampled_normals = np.vstack([sparse_normals, final_valid_normals])
    upsampled_colors = np.vstack([sparse_colors, final_valid_colors])
    
    upsampled_pcd = o3d.geometry.PointCloud()
    upsampled_pcd.points = o3d.utility.Vector3dVector(upsampled_points)
    upsampled_pcd.normals = o3d.utility.Vector3dVector(upsampled_normals)
    upsampled_pcd.colors = o3d.utility.Vector3dVector(upsampled_colors)
    
    return upsampled_pcd


In [31]:
def cluster_point_cloud_new(outlier_cloud,dist):
    cloud_colors = copy.deepcopy(np.asarray(outlier_cloud.colors).T)
    
    with o3d.utility.VerbosityContextManager(o3d.utility.VerbosityLevel.Error) as cm:
        labels = np.array(outlier_cloud.cluster_dbscan(eps=dist, min_points=10, print_progress=False))

    # Identify the largest cluster
    values, counts = np.unique(labels, return_counts=True)
    ind = np.argmax(counts)
    largest_cluster_label = values[ind]
    #print(f"Largest cluster label: {largest_cluster_label}")

    # Filter points, normals, and colors for the largest cluster
    cloud_xyz = pcd2xyz(outlier_cloud)
    cloud_normals = pcd2normals(outlier_cloud)

    cloud_filtered = cloud_xyz[:, labels == largest_cluster_label]
    normals_filtered = cloud_normals[:, labels == largest_cluster_label]
    colors_filtered = cloud_colors[:, labels == largest_cluster_label]

    # Create a point cloud for the largest cluster
    pcd_filtered_largest_cluster = o3d.geometry.PointCloud()
    pcd_filtered_largest_cluster.points = o3d.utility.Vector3dVector(cloud_filtered.T)
    pcd_filtered_largest_cluster.normals = o3d.utility.Vector3dVector(normals_filtered.T)
    pcd_filtered_largest_cluster.colors = o3d.utility.Vector3dVector(colors_filtered.T)

    #o3d.visualization.draw_geometries([pcd_filtered_largest_cluster])
    return pcd_filtered_largest_cluster

In [ ]:
dpath+"animal_1_1_ds.ply"

In [ ]:
for i in range(len(pcds)):
    o3d.visualization.draw_geometries([pcds[i]])

In [ ]:
o3d.visualization.draw_geometries([ptsdf[0]])

In [4]:

rgb_model_path = '/home/vigir3d/Datasets/cattle_scans/maskrcnn_data/maskrcnn_v2.pth'
depth_model_path = '/home/vigir3d/Datasets/cattle_scans/maskrcnn_data/maskrcnn_depth_best.pth'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loads the maskrcnn trained for depth and rgb
model_rgb = load_maskrcnn_model(rgb_model_path)
model_depth = load_maskrcnn_model(depth_model_path)


transform = T.ToTensor()

In [ ]:
dpath

In [ ]:
pcds[0]

In [5]:
r01 = np.loadtxt(dpath+"r01.txt" )
r12 = np.loadtxt(dpath+"r12_0031.txt")
r32 = np.loadtxt(dpath+"r32_0026.txt")
r52 = np.loadtxt(dpath+"r52_005.txt")
r43 = np.loadtxt(dpath+"r43_0031.txt")

h0 = r12@r01
h1 = r12
h3 = r32
h4 = r32@r43
h5 = r52


NameError: name 'dpath' is not defined

In [6]:
# # transform!
# np.savetxt(dpath+"rq01.txt", r01)
# np.savetxt(dpath+"rq12_0031.txt", r12)
# np.savetxt(dpath+"rq32_0026.txt", r32)
# np.savetxt(dpath+"rq52_005.txt", r52)
# np.savetxt(dpath+"rq43_0031.txt", r43)

In [ ]:
for i in range(len(pcds)):
    o3d.visualization.draw_geometries([pcds[i],ptsdf[i]])

In [ ]:
o3d.visualization.draw_geometries([ptsdf[0]])

In [17]:
# transforms
h0 = r12@r01
h1 = r12
h3 = r32
h4 = r32@r43
h5 = r52

p0 = copy.deepcopy(ptsdf[0]).transform(h0)
p1 = copy.deepcopy(ptsdf[1]).transform(h1)
p3 = copy.deepcopy(ptsdf[3]).transform(h3)
p4 = copy.deepcopy(ptsdf[4]).transform(h4)
p5 = copy.deepcopy(ptsdf[5]).transform(h5)

pcd_a = o3d.geometry.PointCloud()
pcd_a = p0+p1+p3+p4+p5+ptsdf[2]
#o3d.io.write_point_cloud(dpath+"animal_1_1_ds.ply", pcd_f)

In [ ]:
for i in range(len(pcds)):
    o3d.visualization.draw_geometries([pcds[i]])

In [ ]:
o3d.visualization.draw_geometries([d0,d1,d3,d4,d5,pcds[2]])

In [ ]:
o3d.visualization.draw_geometries([p0,p1,p3,p4,p5,ptsdf[2]])

In [ ]:
vs = 0.1
#t01,r01 = register_two_views_teaser(ptsdf[0],ptsdf[1],vs)
#t12,r12 = register_two_views_teaser(ptsdf[1],ptsdf[2],vs) # 0.031
#t32,r32 = register_two_views_teaser(ptsdf[3],ptsdf[2],vs) # 0.026
#t52,r52 = register_two_views_teaser(ptsdf[5],ptsdf[2],0.05)
#t43,r43 = register_two_views_teaser(ptsdf[4],ptsdf[3],vs) # 0.031


In [ ]:
r01 = demo_manual_registration(ptsdf[0],ptsdf[1])

In [7]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])


def pick_points(pcd):
    print("")
    print(
        "1) Please pick at least three correspondences using [shift + left click]"
    )
    print("   Press [shift + right click] to undo point picking")
    print("2) After picking points, press 'Q' to close the window")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  # user picks points
    vis.destroy_window()
    print("")
    return vis.get_picked_points()


def demo_manual_registration(src, tgt):
    print("Demo for manual ICP")
    source = src
    target = tgt
    print("Visualization of two point clouds before manual alignment")
    draw_registration_result(source, target, np.identity(4))

    # pick points from two point clouds and builds correspondences
    picked_id_source = pick_points(source)
    picked_id_target = pick_points(target)
    assert (len(picked_id_source) >= 3 and len(picked_id_target) >= 3)
    assert (len(picked_id_source) == len(picked_id_target))
    corr = np.zeros((len(picked_id_source), 2))
    corr[:, 0] = picked_id_source
    corr[:, 1] = picked_id_target

    # estimate rough transformation using correspondences
    print("Compute a rough transform using the correspondences given by user")
    p2p = o3d.pipelines.registration.TransformationEstimationPointToPoint()
    trans_init = p2p.compute_transformation(source, target,
                                            o3d.utility.Vector2iVector(corr))

    # point-to-point ICP for refinement
    print("Perform point-to-point ICP refinement")
    threshold = 0.03  # 3cm distance threshold
    reg_p2p = o3d.pipelines.registration.registration_icp(
        source, target, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPoint())
    draw_registration_result(source, target, reg_p2p.transformation)
    return reg_p2p.transformation
    print("")